In [1]:
from env.dealers import SimpleDealer
from env.table import Table

from player.agents import SimpleAgent
from player.modules import *
from player.brains import SimpleBrain, NeuralHistoryCompressor

from utils import ConditionalExpectation_Loss

In [2]:
num_players = 9
num_agents = 12
num_cards = 52 + 1
bins = 11
actions_dim = bins + 4
embedding_hidden_dim = 7
feedforward_dim = 24
mlp_hidden = 36
embedding_output_dim = 2 + 5 * embedding_hidden_dim + actions_dim # check that it is equal to 3 + 7 * embedding_hidden_dim
history_compressor_depth = 100
start_credits = 1000

lstm_params = [
    {
        "input_size": embedding_output_dim,
        "hidden_size": actions_dim,
        "num_layers": 1
    },

    {
        "input_size": actions_dim,
        "hidden_size": actions_dim,
        "num_layers": 1
    },

    {
        "input_size": actions_dim,
        "hidden_size": actions_dim,
        "num_layers": 1
    }
]

extractor_parameters = {
    "d_model": embedding_output_dim, 
    "nhead": 4, 
    "num_encoder_layers": 4, 
    "num_decoder_layers": 4, 
    "dim_feedforward": feedforward_dim
}

functions = [Fourier, Fourier, Fourier]
function_args = [
    {"dim_in": embedding_hidden_dim * 3, "depth": 15},
    {"dim_in": embedding_hidden_dim, "depth": 15},
    {"dim_in": embedding_hidden_dim, "depth": 15},
]

agent_args = {
    "start_type": TransformersInput,
    "start_args" : {
        "first_args" : {
            "d_model": embedding_output_dim, 
            "nhead": 4, 
            "num_encoder_layers": 4, 
            "num_decoder_layers": 4, 
            "dim_feedforward": feedforward_dim
        }, 
        "second_args" : {
            "d_model": 2 * actions_dim, 
            "nhead": 5, 
            "num_encoder_layers": 4, 
            "num_decoder_layers": 4, 
            "dim_feedforward": feedforward_dim
        }
    },

    "module_args" : {
        "First_Numbered_MLP" : {
            "dim_in" : 2 * actions_dim + embedding_output_dim, 
            "hidden_dim" : mlp_hidden,
            "dim_out" : actions_dim, 
            "depth" : 3, 
            "num_agents": num_agents
        },
        
        "FourierOutput" : {
            "dim_in" : actions_dim, 
            "dim_encoder_output": 2 * actions_dim + embedding_output_dim,
            "hidden_dim" : mlp_hidden,
            "shapes" : [[embedding_hidden_dim * 3, 15], [embedding_hidden_dim, 15], [embedding_hidden_dim, 15]], 
            "depth" : 3,
            "num_agents": num_agents
        }
    },

    "modules" : {
        "First_Numbered_MLP" : Numbered_MLP,
        "FourierOutput" : FourierOutput
    }
}

In [3]:
table = Table(num_players, bins = bins)

loss = ConditionalExpectation_Loss(start_credits, function_args, functions)
memory = NeuralHistoryCompressor(num_agents, history_compressor_depth, lstm_params, extractor_parameters, train_freq = 5)
embedding = SimpleEmbedding(num_cards, embedding_hidden_dim, num_players, start_credits)

brain = SimpleBrain(num_agents, loss, memory, SimpleAgent, embedding, agent_args)

In [4]:
dealer = SimpleDealer(table, brain)

In [5]:
n_games = 2

dealer.game(n_games, num_players, 180, 1)

game  0  is finished!
Warning! Some player didn`t any action!
game  1  is finished!
